# Player Statistics

This notebook attempts to build an ML model for predicting the position of a player given their statistics.

Potential Resources to download stats:

* https://www.basketball-reference.com/leagues/NBA_2023_per_game.html (Download as many seasons as possible). For each download, need to read the file in, and collate player statistics such that players who played on multiple teams that season have their stats collated.